In [1]:
import gtfs_functions as gtfs
import matplotlib
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import random
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# cluster visualizer
#%matplotlib inline
#from yellowbrick.cluster import KElbowVisualizer 

# sklearn kmeans
from sklearn.metrics.cluster import contingency_matrix

# pyclustering kmeans
from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.kmedians import kmedians
from pyclustering.utils.metric import distance_metric
from pyclustering.cluster.center_initializer import random_center_initializer
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.cluster.encoder import type_encoding
from pyclustering.cluster.encoder import cluster_encoder
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from numpy import unique
from numpy import where
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.mixture import GaussianMixture

import csv

In [3]:
#We import the schedule

path_gtfs ='C:/Users/Maison/Documents/INFO-H423/Projet Data Mining/GTFS3sep/gtfs3sept.zip'

routes, stops, stop_times, trips, shapes = gtfs.import_gtfs(path_gtfs,busiest_date=False)

In [4]:
#We import the calendar by hand 

calendardf = pd.read_table('calendar.txt',sep=',')          #export the calendar file into data frame
calendarDatesdf = pd.read_table('calendar_dates.txt',sep=',')  #also export the exceptions

In [5]:
def FromDayToService(date):
    '''Determine all the service_id concerned by a date, given in the format date=('20210907','tuesday')
        it first take a look at every regular service_id then examine the exceptions. 
        Returns a list of service_id of type string'''

    ListServiceId=[]
    for i in range(len(calendardf)):
        startdate,endate=calendardf.iloc[i,[8,9]]
        if int(date[0]) in range(int(startdate),int(endate)+1):
            if calendardf.loc[i,date[1]]==1:
                #It appends every service_id which contains the date, and for which there is a '1'
                # under right day in the week
                ListServiceId.append(str(calendardf.iloc[i,0]))     

    #Now we have to take a look a the exceptions
    #We selections all the exceptions in calendarDatesdf that concerns the date in the input
    ModifServiceId= calendarDatesdf[calendarDatesdf['date']==int(date[0])][['service_id','exception_type']]
    #print('Number of service_id involved in modifications: '+ str(len(ModifServiceId)))
    #for every modif we check the kind of exception
    for modif in ModifServiceId.iterrows():  
        service = str(modif[1]['service_id'])
        exception = modif[1]['exception_type']
        if exception == 1:
            #if the exception is 1, it means that we should change to a 1 the cell related to the date in
            # input, in the row equal to service in calendar
            ListServiceId.append(service)
            #print('ServiceId: '+str(service)+' correclty added' )
        elif (exception == 2) & (service in ListServiceId):
            #if the exception is 2, it means that we should change to a 0 the cell related to the date in
            # input, in the row equal to service in calendar
            #Since this service_id must have been selected in the first loop (in Calendar), we need to remove it
            ListServiceId.remove(service)
            #print('ServiceId: '+str(service)+' correclty removed' )
        elif (exception == 2) & (service not in ListServiceId):
            #if the exception is 2 but we haven't found the service id in the first loop, this is weird
            #In principle this should not happend, but we're never too carefull
            print('Warning : exception 2 but no service_id = ' +service+ ' found in calendar')
    return (ListServiceId)

def FromLineIdtoRouteId(LineId):
    
    '''Takes a lineid in argument a returns the correspondent routeid, since stop_times only 
        contains route_id'''

    routeId = routes[routes['route_short_name']==LineId]['route_id'].iloc[0]
    return routeId

def ScheduledTime(date,routeid,stopId):

    '''Takes a triplet (date,routeid,stopId) with date in the format date=('20210907','tuesday')
        and returns the schedule for that triplet. The schedule is composed as follows: if we are in a period of
        punctuality then returns the (arrivaltime,true), and if we are in regularity, 
        returns (TimeOfWaitFromPreviousVehicle,False), all in one list '''

    ListOfService = pd.Series(FromDayToService(date))  #List of service_id tha apply for that day
    #RouteId = FromLineIdtoRouteId(lineId)               #We need the routeid to seek in the GTFS
    #We select the relevant rows in GTFS stop_times
    Scheduledf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==routeid)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    ArrivalTime = Scheduledf['arrival_time'].values
    #print('Number of service_id involved: '+ str(len(ListOfService)))
    #We compute every wainting time to determine punctuality of regularity (< or > 720 s)
    #Schedule = list(ArrivalTime[1:]-ArrivalTime[:-1])
    IntervalTime = list(ArrivalTime[1:]-ArrivalTime[:-1])
    #Schedule.insert(0,ArrivalTime[0])
    #True if punctuality, False if regularity
    Punctuality = np.array(IntervalTime)>720 
    #Ultimately, if we are a punctuality, we want the arrival time and if we are in regularity, we want the 
    #time bewteen two vehicles
    for i,punctuality in enumerate(Punctuality):
        if punctuality:
            IntervalTime[i]=ArrivalTime[i]
            #if (0<i<(len(Punctuality)-1)):
                #if ~(Punctuality[i-1]) & ~(Punctuality[i+1]):
                    #Punctuality[i]=False
            #else:
                
 
    #Schedule = list(zip(Schedule,Punctuality))
    #The format is a list of tuples
    if len(IntervalTime)!=len(ArrivalTime[:-1]):
        print('Attention: la longueur de Schedule et ArrivalTime ne match pas dans ScheduledTime ')
    return IntervalTime,Punctuality,ArrivalTime[:-1]

def RouteIdToLineId(routeId):

    '''Takes a routeid in argument a returns the correspondent lineid'''

    lineId = routes[routes['route_id']==routeId]['route_short_name'].iloc[0]
    return lineId

def TroncatedScheduleRegularity(date,routeid,stopId):

    '''Takes a triplet (date,routeid,stopId) in the format date=('20210907','tuesday') and returns the IntervalTime 
        and ArrivalTime cleared from the ponctuality point'''
    ScheduleNonCleared = ScheduledTime(date,routeid,stopId)
    TroncatedInterval = [intervaltime for i,intervaltime in enumerate(ScheduleNonCleared[0]) if not(ScheduleNonCleared[1][i])]
    TroncatedArrival = [arrivaltime for i,arrivaltime in enumerate(ScheduleNonCleared[2]) if not(ScheduleNonCleared[1][i])]
    if len(TroncatedInterval)!= len(TroncatedArrival):
        print('Attention: la longueur de TroncatedInterval et TroncatedArrival ne match pas dans TroncatedScheduleRegularity ')
    return TroncatedInterval,TroncatedArrival

def ScheduledTimeCSV(date,routeid,stopId):

    '''Takes a triplet (date,lineId,stopId) with date in the format date=('20210907','tuesday')
        and returns the schedule for that triplet. THE DIFFERENCE WITH ScheduledTime IS THAT WE DONT
        NEED THE PUNCTUALITY'''

    ListOfService = pd.Series(FromDayToService(date))              #We need the routeid to seek in the GTFS
    #We select the relevant rows in GTFS stop_times
    Scheduledf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==routeid)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    directionId = Scheduledf['direction_id'].unique()
    ArrivalTime = Scheduledf['arrival_time'].values
    

    return ArrivalTime,directionId

In [6]:
StopTimesGroupTrip = stop_times.groupby('trip_id')[['stop_sequence']].mean().round()
StopTimesGroupTrip['stop_sequence']=StopTimesGroupTrip['stop_sequence'].astype(int)

StopTimesOnlyMeanSeq = pd.merge(stop_times, StopTimesGroupTrip, how='right', on= ['trip_id','stop_sequence'])
StopTimesOnlyMeanSeq = StopTimesOnlyMeanSeq[['route_id','stop_sequence','stop_id','direction_id']]
DicOfMiddleStop={}

for route in StopTimesOnlyMeanSeq['route_id'].unique():
    StopTimeMeanSeqRoute = StopTimesOnlyMeanSeq[StopTimesOnlyMeanSeq['route_id']==route]
    directionIds = StopTimeMeanSeqRoute['direction_id'].unique()
    for direction in directionIds:
        #Ici je prends la première valeure que je trouve qui a la bonne route et direction
        # car il peut y avoir plusieurs candidat Middle stop pour chaque (ligne,direction)
        # ici je prend le premier parce que j'en ai très exactement rien à fouttre
        stop = StopTimeMeanSeqRoute[StopTimeMeanSeqRoute['direction_id']==direction]['stop_id'].values[0]
        value = str(route)+':'+str(direction)
        if DicOfMiddleStop.get(stop) is None:
            DicOfMiddleStop[stop]=value


#Format du dico : 'middle_stop_id': 'routeid:direction'    Ex: '5510': '41:0'

#Ici c'est juste pour lui dire quelle date prendre en exemple si on dit 'weekday', 'satuday', etc...
DicOfWeek = {'weekday':('20210915','wednesday'),'saturday':('20210918','saturday'),'sunday':('20210919','sunday')}

#liste des middle stop
ListOfMiddleStop= list(DicOfMiddleStop.keys())

def get_key(val):
    for key, value in DicOfMiddleStop.items():
        if val == value:
            return key


In [7]:
ListOfStopId  = list(stop_times['stop_id'].unique())
DicOfStopInRoute = {}
for route in routes['route_id']:
    ListOfStop = [stop for stop in stop_times[stop_times['route_id']==route]['stop_id'].unique()]
    DicOfStopInRoute[route]=ListOfStop

Algo de Cluster de Gulli

In [8]:
distance_measures = {'euclidean': 0, 'squared euclidean': 1, 'manhattan': 2, 'chebyshev': 3, 
                    'canberra': 5, 'chi-square': 6}

routeidTram = ['3','10','6','11','12','25','76','41','40','15','26','9','42','14','16','7','75','8']
routeidMetro = ['2','4','1','3']
routeidBus = ['5', '13', '17', '18', '19', '20', '21', '22', '23', '24', '27', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '43', '44', '45', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '77', '78', '79', '80', '81', '82', '84', '85', '86', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97']



def ClusteringOfData(dist_measure,Data,DataDelta,routeid,dayoftheweek):

    #il faut sortir routeidMetro etc de la boucle
    Clusters = []
    routeidMetro = ['2','4','1','3']
    routeidTram = ['3','10','6','11','12','25','76','41','40','15','26','9','42','14','16','7','75','8']

    #delete if len(data)<10

    if routeid in routeidMetro:
        if dayoftheweek == 'weekday':
            nclust = 4
            nY = 1000
            coefCent = 1.05
        else:
            nclust = 3
            nY = 10000
            coefCent = 1.05
    elif routeid in routeidTram:
        if dayoftheweek == 'weekday':
            #print(len(Data))
            nclust = len(Data)//35
            nY = 1
            coefCent = 1
        else:
            #print(len(Data))
            nclust = len(Data)//25
            nY = 1000
            coefCent = 1
    else:
        if dayoftheweek == 'weekday':
            #print(len(Data))
            nclust = len(Data)//35
            nY = 1000
            coefCent = 1
        else:
            #print(len(Data))
            nclust = len(Data)//25
            nY = 1000
            coefCent = 1
    
    if nclust == 1 or nclust == 0:
        return(Clusters,nclust)

    maxX = np.max(DataDelta.iloc[:,:1])*1
    maxY = np.max(DataDelta.iloc[:,1:])*nY
    DataDelta.iloc[:,:1] = DataDelta.iloc[:,:1]/maxX
    DataDelta.iloc[:,1:] = DataDelta.iloc[:,1:]/maxY

    initial_centers = kmeans_plusplus_initializer(DataDelta.values, nclust,amount_candidates = 4).initialize()
    instanceKm = kmeans(DataDelta.values, initial_centers=initial_centers, metric=distance_metric(dist_measure))
    instanceKm.process()
    Centers = instanceKm.get_centers()

    for center in Centers:
        center[0] = center[0]*coefCent

    instanceKmedians = kmedians(DataDelta.values, initial_medians=Centers,tolerance = 0.001)
    instanceKmedians.process()
    Clusters = instanceKmedians.get_clusters() 


    return(Clusters,nclust)

Cellule du Mercredi à exécuter 

In [9]:

#Runtime ~ 35min 

#Cellule pour mecredi 15/09

header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6']

with open('TimeGroupWednesday20210915AllStop.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            DataTemp = TroncatedScheduleRegularity(('20210915','wednesday'),route,stop)
            IntervalTime = DataTemp[0]
            ArrivalTime = DataTemp[1]
            Data = pd.DataFrame({'ArrivalTime':ArrivalTime, 'TimeWaited':IntervalTime})
            DataDelta= pd.DataFrame({'ArrivalTime_i':ArrivalTime[:-1], 'TimeWaited':ArrivalTime[1:]})
            if len(Data)>10:
                #ClusteringOfData(dist_measure,nclust,Data,DataDelta,routeid,dayoftheweek):
                ClustersOutput = ClusteringOfData(distance_measures['manhattan'],Data,DataDelta,route,'weekday')
                Clusters = ClustersOutput[0]
                NumCluster = ClustersOutput[1]
                Clusters.sort(key=lambda x: x[0])
                row = [route,stop]
                for cluster in Clusters:
                    #MeanIntervalTimeInCluster = Data.iloc[cluster[0]:cluster[-1],1].mean()
                    #Warning we need to invert since every cluster is in other order
                    SWT = Data.iloc[cluster[-1]:cluster[0],1].values
                    if sum(SWT)!=0:
                        SWT = sum(SWT**2)/(2*sum(SWT))
                    elif sum(SWT)==0:
                        print('Problem with SWT on stop: '+str(stop))
                        SWT='Problem with SWT'
                    #rangeCluster = str(Data.iloc[cluster[0],0])+'::'+str(Data.iloc[cluster[-1],0])+'::'+str(MeanIntervalTimeInCluster)
                    rangeCluster = str(Data.iloc[cluster[-1],0])+'::'+str(Data.iloc[cluster[0],0])+'::'+str(SWT)
                    row.append(rangeCluster)
                for i in range(NumCluster,6):
                    row.append('0::0::0')
                writer.writerow(row)
        k+=1
        if k%10==0:
            print(str(k)+' routes encodées')

Cellule du Vendredi à exécuter 

In [ ]:
#Runtime ~ 35min 

#Cellule pour vendredi 17/09

header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6']

with open('TimeGroupFriday20210917AllStop.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            DataTemp = TroncatedScheduleRegularity(('20210917','friday'),route,stop)
            IntervalTime = DataTemp[0]
            ArrivalTime = DataTemp[1]
            Data = pd.DataFrame({'ArrivalTime':ArrivalTime, 'TimeWaited':IntervalTime})
            DataDelta= pd.DataFrame({'ArrivalTime_i':ArrivalTime[:-1], 'TimeWaited':ArrivalTime[1:]})
            if len(Data)>10:
                #ClusteringOfData(dist_measure,nclust,Data,DataDelta,routeid,dayoftheweek):
                ClustersOutput = ClusteringOfData(distance_measures['manhattan'],Data,DataDelta,route,'weekday')
                Clusters = ClustersOutput[0]
                NumCluster = ClustersOutput[1]
                Clusters.sort(key=lambda x: x[0])
                row = [route,stop]
                for cluster in Clusters:
                    #MeanIntervalTimeInCluster = Data.iloc[cluster[0]:cluster[-1],1].mean()
                    #Warning we need to invert since every cluster is in other order
                    SWT = Data.iloc[cluster[-1]:cluster[0],1].values
                    if sum(SWT)!=0:
                        SWT = sum(SWT**2)/(2*sum(SWT))
                    elif sum(SWT)==0:
                        print('Problem with SWT on stop: '+str(stop))
                        SWT='Problem with SWT'
                    #rangeCluster = str(Data.iloc[cluster[0],0])+'::'+str(Data.iloc[cluster[-1],0])+'::'+str(MeanIntervalTimeInCluster)
                    rangeCluster = str(Data.iloc[cluster[-1],0])+'::'+str(Data.iloc[cluster[0],0])+'::'+str(SWT)
                    row.append(rangeCluster)
                for i in range(NumCluster,6):
                    row.append('0::0::0')
                writer.writerow(row)
        k+=1
        if k%10==0:
            print(str(k)+' routes encodées')

Cellule du samedi à exécuter

In [12]:
#Runtime ~ 35min 

#Cellule pour samedi 18/09

header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6']

with open('TimeGroupSaturday20210918AllStop.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            DataTemp = TroncatedScheduleRegularity(('20210918','saturday'),route,stop)
            IntervalTime = DataTemp[0]
            ArrivalTime = DataTemp[1]
            Data = pd.DataFrame({'ArrivalTime':ArrivalTime, 'TimeWaited':IntervalTime})
            DataDelta= pd.DataFrame({'ArrivalTime_i':ArrivalTime[:-1], 'TimeWaited':ArrivalTime[1:]})
            if len(Data)>10:
                #ClusteringOfData(dist_measure,nclust,Data,DataDelta,routeid,dayoftheweek):
                ClustersOutput = ClusteringOfData(distance_measures['manhattan'],Data,DataDelta,route,'saturday')
                Clusters = ClustersOutput[0]
                NumCluster = ClustersOutput[1]
                Clusters.sort(key=lambda x: x[0])
                row = [route,stop]
                for cluster in Clusters:
                    #MeanIntervalTimeInCluster = Data.iloc[cluster[0]:cluster[-1],1].mean()
                    #Warning we need to invert since every cluster is in other order
                    SWT = Data.iloc[cluster[-1]:cluster[0],1].values
                    if sum(SWT)!=0:
                        SWT = sum(SWT**2)/(2*sum(SWT))
                    elif sum(SWT)==0:
                        print('Problem with SWT on stop: '+str(stop))
                        SWT='Problem with SWT'
                    #rangeCluster = str(Data.iloc[cluster[0],0])+'::'+str(Data.iloc[cluster[-1],0])+'::'+str(MeanIntervalTimeInCluster)
                    rangeCluster = str(Data.iloc[cluster[-1],0])+'::'+str(Data.iloc[cluster[0],0])+'::'+str(SWT)
                    row.append(rangeCluster)
                for i in range(NumCluster,6):
                    row.append('0::0::0')
                writer.writerow(row)
        k+=1
        if k%10==0:
            print(str(k)+' routes encodées')

Problem with SWT on stop: 8162


Cellule du Dimanche à exécuter

In [13]:
#Runtime ~ 35min 

#Cellule pour dimanche 19/09

header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6']

with open('TimeGroupSunday20210919AllStop.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            DataTemp = TroncatedScheduleRegularity(('20210919','sunday'),route,stop)
            IntervalTime = DataTemp[0]
            ArrivalTime = DataTemp[1]
            Data = pd.DataFrame({'ArrivalTime':ArrivalTime, 'TimeWaited':IntervalTime})
            DataDelta= pd.DataFrame({'ArrivalTime_i':ArrivalTime[:-1], 'TimeWaited':ArrivalTime[1:]})
            if len(Data)>10:
                #ClusteringOfData(dist_measure,nclust,Data,DataDelta,routeid,dayoftheweek):
                ClustersOutput = ClusteringOfData(distance_measures['manhattan'],Data,DataDelta,route,'sunday')
                Clusters = ClustersOutput[0]
                NumCluster = ClustersOutput[1]
                Clusters.sort(key=lambda x: x[0])
                row = [route,stop]
                for cluster in Clusters:
                    #MeanIntervalTimeInCluster = Data.iloc[cluster[0]:cluster[-1],1].mean()
                    #Warning we need to invert since every cluster is in other order
                    SWT = Data.iloc[cluster[-1]:cluster[0],1].values
                    if sum(SWT)!=0:
                        SWT = sum(SWT**2)/(2*sum(SWT))
                    elif sum(SWT)==0:
                        print('Problem with SWT on stop: '+str)
                        SWT='Problem with SWT'
                    #rangeCluster = str(Data.iloc[cluster[0],0])+'::'+str(Data.iloc[cluster[-1],0])+'::'+str(MeanIntervalTimeInCluster)
                    rangeCluster = str(Data.iloc[cluster[-1],0])+'::'+str(Data.iloc[cluster[0],0])+'::'+str(SWT)
                    row.append(rangeCluster)
                for i in range(NumCluster,6):
                    row.append('0::0::0')
                writer.writerow(row)
        k+=1
        if k%10==0:
            print(str(k)+' route encodées')